<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AIX_1A_ml_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#Modell: Ett enkelt CNN (Convolutional Neural Network)

###Samma problem som för **AIX_1_machine_learning** men nu med en bättre modell. Vi kommer att nå en noggrannhet på c:a 99% d.v.s. felet är halverat. Den här modellen tar längre tid att träna därför kommer vi att använda en GPU för träningen. Så här gör du för att allokera en GPU: **Anslut->Ändra körningstyp** och välj en GPU.
---

---
# Våra data: Dataset MNIST - handskrivna siffror 0-9.
---



In [ ]:
# Läs in dataset MNIST från Keras

from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
# träningsdata består av 60,000 bilder med 28x28 pixlar

train_images.shape

In [ ]:
# till varje bild finns en label (0-9) som talar om siffran

len(train_labels)

In [ ]:
# vi använder testdata (10,000 bilder) för att utvärdera modellen

test_images.shape

In [ ]:
# även för varje testbild finns en label (0-9)

len(test_labels)

---
#En bättre modell: Ett enkelt CNN
---
Noggrannhet på testdata efter ungefär 5 epoker: 99% d.v.s. en halvering av felet.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
# kompilera modellen
model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [ ]:
# träna modellen med 64 bilder i taget. Kör igenom alla data 5 gånger
model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

---
#Felklassificerade bilder
---
D.v.s. bilder där prediktion och label är olika.


In [ ]:
# Funktion för att visa felklassificerade bilder
import numpy as np
import matplotlib.pyplot as plt

def bad_pred(data, labels, maxfel):
    ncorr = 0
    nfel = 0
    for i in range(len(data)):
        image = data[i][np.newaxis]
        predikt = model.predict(image, verbose=0).argmax()
        if (predikt != labels[i]):
            nfel = nfel + 1
            print('-------------------------------------------------------')
            plt.figure(figsize=(3, 2))
            plt.imshow(data[i], cmap='gray')
            plt.show()
            print("Bild nr:" + str(i))
            print("Label:" + str(labels[i]))
            print("Prediktion:" + str(predikt))
            if (nfel >= maxfel):
                break
    print("Antal fel:" + str(nfel))


In [ ]:
# Visa 10 felklassificerade bilder
bad_pred(test_images, test_labels, 10)